In [77]:
import pandas as pd
import re
from nltk.cluster import KMeansClusterer
import nltk
import numpy as np
import spacy
from sklearn.cluster import KMeans
from tqdm import tqdm
import requests

In [60]:
# load file

locations = pd.read_csv('./data/librettos_theaters.csv')
locations['id'] = [int(re.findall(r'\d+', s)[0]) for s in locations.file_name]
locations = locations.drop(columns=['Unnamed: 0'])
locations.sample(5)

,file_name,title,date,coperta,pot_city_name,coperta_appended,city_name,latitude,longitude,pot_city_name_fuzzy,...,location_vec_method_1,predicted_cluster_method_1,inferred_latitude,inferred_longitude,title_opera_method_1,title_opera_method_2,title_vec_method_1,predicted_title_cluster_method_1,composer,inferred_title
385,data_773.json,La forza delle donne dramma giocoso per musica...,1781,"['ANFOSS', 'es', 'Bi', '«eae', 'Qa', 'LiA', 'F...","['bologna', 'bologna']",0,bologna,44.49381,11.33875,"['bologna', 'bologna']",...,[ 4.325851 -0.9764209 3.5252006 -2.955655...,34,44.4698796,11.1005849,La forza delle donne,La forza delle donne,[-4.1893911e-01 1.3452771e-01 -1.9447869e+00 ...,195,Not found,La forza delle donne
931,data_490.json,Il trionfo della costanza in Statira vedova d'...,1730,"['IL', 'TRIONFO', 'DELLA', 'COSTANZA', 'IN', '...","['venezia', 'venezia', 'venetia']",0,venezia,45.43713,12.33265,"['venezia', 'venezia', 'venetia']",...,[ 3.8287554 1.0209062 1.5838853 -2.686141...,20,Not found,Not found,Il trionfo della costanza in Statira vedova d'...,Il trionfo della costanza in Statira vedova d'...,[ 1.5186474 2.184085 0.31559575 -1.018549...,809,Not found,Il trionfo della costanza in Statira vedova d'...
437,data_5.json,"Adelaide, dramma per musica da rappresentarsi ...",1729,"['Guawom', 'ADELA', 'TDS', '§', 'Dramma', 'per...","['marian', 'reggio', 'venezia', 'venetia']",0,marian,45.69651,9.18180,"['marian', 'reggio', 'venezia', 'venetia']",...,[ 2.348701 -0.9958396 2.3199444 -2.183356...,16,45.776982149999995,8.882520992545153,Adelaide,Adelaide,[ 4.5501895e+00 1.3058958e+00 4.7333851e+00 ...,89,Not found,Adelaide
1098,data_1097.json,Una cosa rara ossia bellezza ed onestà dramma ...,1790,"['UNA', 'COSA', 'RARA', 'BELLEZZA', 'ED', 'ONE...","['milano', 'milano']",0,milano,45.46427,9.18951,"['milano', 'milano']",...,[-1.0068026 0.18633239 1.6841553 -3.859258...,130,Not found,Not found,Una cosa rara ossia bellezza ed onestà,Una cosa rara ossia bellezza ed onestà,[-0.08177555 2.2459939 0.01861072 0.857303...,63,Not found,Una cosa rara o sia bellezza ed onestà
463,data_496.json,Il trionfo di Clelia dramma per musica rappres...,1762,"['HASSE', 'Teantinticu', '162', 'fl', '.', 'IL...","['napoli', 'napoli']",0,napoli,40.85216,14.26811,"['napoli', 'napoli']",...,[ 1.8149211e+00 -9.6280670e-01 4.1678529e+00 ...,1,Not found,Not found,Il trionfo di Clelia,Il trionfo di Clelia,[ 1.01764464e+00 2.83364677e+00 2.36408758e+...,647,Not found,Il trionfo di Clelia


In [85]:
expr = r'''(?i)modi sacri |(?i)rappresentato con |(?i)dramma|(?i)drama|(?i)melodramma|(?i)melodrama|(?i)melo-dramma|(?i)oratorio|(?i)favola pastorale|(?i)fauola musicale|(?i)burletta in musica|(?i)componimento sagro|(?i)azione musicale|(?i)componimento sacro|(?i)per musica|(?i)opera|(?i)tragedia|(?i)spettacolo fantastico|(?i)scena in versi|(?i)fauola tragicomica|(?i)fauola pastorale|(?i)diuertimento comico|(?i)azione romantica|(?i)farsa|(?i)commeddia|(?i)commedia|(?i)operetta|(?i)festa|(?i)cantata|(?i)intermezzo in musica|(?i)intermezzi|(?i)favola in prosa|(?i)componimento sagro|(?i)componimento|(?i)azione sacra|(?i)actio sacra|(?i)fiaba|(?i)libretto fantastico|(?i)libretto|(?i)fauola|(?i)favola'''#|([a-rt-z])\. ''' 

locations['title_opera_method_1'] = [re.split(expr,s)[0].strip().strip(',').strip('.')
                                     if len(re.split(expr,s)) > 0 else 'Not found'
                                     for s in locations.title]

locations.title_opera_method_1.value_counts()

                                11
Antigona                         7
Il geloso in cimento             7
Artaserse                        7
La vera costanza                 7
                                ..
Antigona in Tebe                 1
Attalo re di Bitinia             1
Arsinoe                          1
Ivanhoe                          1
L' Antigona delusa d'Alceste     1
Name: title_opera_method_1, Length: 882, dtype: int64

In [86]:
nlp = spacy.load("it_core_news_sm")
locations['title_opera_method_2'] = [[ent.text
                                      for ent in nlp(title).ents
                                      ]
                                      for title in locations.title_opera_method_1 
                                      ]
locations['title_opera_method_2'] = [loc[0] if len(loc)>0 else 'Not found' 
                                       for loc in locations['title_opera_method_2']]

locations.title_opera_method_2.value_counts()

Not found           434
Antigona              9
Ciro                  7
Ezio                  7
Olimpiade             7
                   ... 
Ninfa                 1
Giovanna              1
Statira               1
Emma d'Antiochia      1
Artigiani             1
Name: title_opera_method_2, Length: 497, dtype: int64

In [87]:
locations.title_opera_method_1 = ['Not found' if loc == '' else loc for loc in locations.title_opera_method_1]
locations['title_vec_method_1'] = [np.mean(np.array([token.vector for token in nlp(loc)]), axis=0) for loc in locations.title_opera_method_1]
print('Processed the set')

NUM_CLUSTERS = 830
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=0, max_iter=500, verbose=10000).fit(locations['title_vec_method_1'].tolist())

Processed the set
Initialization complete
Iteration 0, inertia 2499.537226728275
Iteration 1, inertia 1482.8240891080434
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 2496.715454328499
Iteration 1, inertia 1538.2681561161999
Iteration 2, inertia 1525.4006110157168
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2615.03695093888
Iteration 1, inertia 1518.4422487361999
Iteration 2, inertia 1505.5747036357166
Converged at iteration 2: strict convergence.
Initialization complete
Iteration 0, inertia 2498.0633885446205
Iteration 1, inertia 1444.447001965397
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 2515.943478892079
Iteration 1, inertia 1487.2245884530928
Converged at iteration 1: strict convergence.
Initialization complete
Iteration 0, inertia 2523.9748321513143
Iteration 1, inertia 1465.8129146482336
Converged at iteration 1: strict convergence.
Initiali

In [88]:
locations['predicted_title_cluster_method_1'] = locations['title_vec_method_1'].apply(lambda x: kmeans.predict([x.tolist()])[0])
locations['predicted_title_cluster_method_1']

0       364
1       440
2       695
3       388
4        60
       ... 
1105     70
1106    777
1107    276
1108    518
1109     18
Name: predicted_title_cluster_method_1, Length: 1110, dtype: int64

In [89]:
cluster_title = {}
for i, cluster in enumerate(locations.predicted_title_cluster_method_1):
    if cluster in cluster_title.keys():
        if locations.title_opera_method_1[i] != 'Not found':
            cluster_title[cluster].append(locations.title_opera_method_1[i])
    else:
            cluster_title[cluster] = [locations.title_opera_method_1[i]]

In [90]:
cluster_title

{364: ['Cesare in Egitto', 'Cesare in Egitto'],
 440: ['Fernando nel Messico'],
 695: ['Adelaide di Borgogna'],
 388: ['S. Elena al Calvario', 'S. Elena al Calvario', 'S. Elena al Calvario'],
 60: ['La vera costanza',
  'La vera costanza',
  'La vera costanza',
  'La vera costanza',
  'La vera costanza',
  'La vera costanza',
  'La vera costanza'],
 228: ['La donna di genio volubile', 'La donna di genio volubile'],
 671: ["L' occasione fa il ladro burletta"],
 221: ["L' Olimpiade",
  "L' Olimpiade",
  "L' Alcibiade",
  "L' Olimpiade",
  "L' Olimpiade"],
 67: ["L'arbore di Diana a comic"],
 2: ["L' amante anonimo"],
 120: ['I Capuleti e i Montecchi'],
 214: ["L' incognita perseguitata",
  "L' incognita perseguitata",
  "L' incognita perseguitata",
  "L' incognita perseguitata",
  "L' incognita perseguitata"],
 88: ['Una cosa rara, ossia Bellezza, ed Onestà',
  'Una cosa rara, ossia Bellezza, ed Onestà',
  'Una cosa rara, ossia Bellezza, de [!] onestà'],
 159: ['Makmus'],
 115: ['Tancred

In [91]:
for k in cluster_title.keys():
    cluster_title[k] = max(set(cluster_title[k]), key=cluster_title[k].count)
    
cluster_title

{364: 'Cesare in Egitto',
 440: 'Fernando nel Messico',
 695: 'Adelaide di Borgogna',
 388: 'S. Elena al Calvario',
 60: 'La vera costanza',
 228: 'La donna di genio volubile',
 671: "L' occasione fa il ladro burletta",
 221: "L' Olimpiade",
 67: "L'arbore di Diana a comic",
 2: "L' amante anonimo",
 120: 'I Capuleti e i Montecchi',
 214: "L' incognita perseguitata",
 88: 'Una cosa rara, ossia Bellezza, ed Onestà',
 159: 'Makmus',
 115: 'Tancredi',
 402: 'Gli amanti alla prova',
 654: "L' Ernelinda",
 532: 'Antigona in Tebe',
 538: 'La falsirena',
 608: 'La donna innamorata',
 128: 'La moglie capricciosa',
 286: 'Agesilao',
 197: 'Leonora',
 528: 'Il Filindo, pastorale eroica',
 93: 'Farnace',
 541: 'Elena e Malvina',
 536: 'Le fortune di Rodope e Damira',
 100: 'Amleto',
 38: 'Tebaldo e Isolina',
 90: 'Il geloso in cimento',
 522: 'Ama più chi men si crede',
 396: "L'impresario burlato",
 264: 'Eugenia',
 10: 'Siroe re di Persia',
 46: "L' innocenza rispettata",
 76: 'Motezuma',
 19: 

In [92]:
locations['inferred_title'] = [cluster_title[cluster] for cluster in locations.predicted_title_cluster_method_1]
locations.inferred_title.value_counts()

Not found                        11
La vera costanza                  7
Artaserse                         7
Antigona                          7
Il geloso in cimento              7
                                 ..
Giocasta regina d'Armenia         1
Le fortune di Rodope e Damira     1
I due gobbi                       1
Arrenione                         1
Medea e Giasone                   1
Name: inferred_title, Length: 830, dtype: int64

In [93]:
locations.to_csv('./data/librettos_theaters.csv')
locations.to_pickle('./data/librettos_theaters.pkl')

In [150]:
S = requests.Session()
URL = "https://it.wikipedia.org/w/api.php"
SEARCHPAGE = locations['inferred_title'][1]
print(SEARCHPAGE)
PARAMS = {
    "action": "query",
    "format": "json",
    "list": "search",
    "srsearch": SEARCHPAGE
}

R = S.get(url=URL, params=PARAMS)
DATA = R.json()
DATA['query']['search'][0]

Fernando nel Messico


{'ns': 0,
 'title': 'Fernando Colunga',
 'pageid': 6051246,
 'size': 8393,
 'wordcount': 911,
 'snippet': '<span class="searchmatch">Fernando</span> Colunga, all\'anagrafe <span class="searchmatch">Fernando</span> Colunga Olivares (Città del <span class="searchmatch">Messico</span>, 3 marzo 1966), è un attore messicano di telenovele. <span class="searchmatch">Fernando</span> è figlio dell\'ingegnere',
 'timestamp': '2020-10-07T03:14:43Z'}

In [107]:
locations['title_mediawiki_pageid'] = [S.get(url=URL, params={"action": "query",
                                                              "format": "json",
                                                              "list": "search",
                                                              "srsearch": title}).json()
                                            ['query']['search'][0]['pageid'] 
                                       if len(S.get(url=URL, params={"action": "query",
                                                                     "format": "json",
                                                                     "list": "search",
                                                                     "srsearch": title}).json()
                                              ['query']['search']) > 0
                                       else
                                           'Not found'
                                       for title in locations.inferred_title
                                       ]
locations['title_mediawiki_pageid']

0       7673260
1       6051246
2        746430
3       4768245
4       5274786
         ...   
1105    2725736
1106    1281516
1107     560173
1108     806397
1109     692436
Name: title_mediawiki_pageid, Length: 1110, dtype: object

In [127]:
locations.to_csv('./data/librettos_theaters.csv', sep='\t')